(ortools-number-link)=
# ナンバーリンクパズル

[前章](z3-number-link)では、Z3を使用してナンバーリンクパズルを解きました。Z3には回路を作成するための専用の制約条件がないため、開発者は自分で類似の制約条件を作成する必要があります。一方、本章では、CP-SATの `add_circuit()` を利用し、回路の制約条件を簡単に設定することで、ナンバーリンクパズルを解く方法を紹介します。  

In [11]:
import numpy as np
from ortools.sat.python import cp_model
from helper.ortools import get_circuit

In [12]:
def load_puzzle(filename):
    with open(filename) as f:
        puzzle = f.read()
        board = np.array([[int(x, 16) for x in row] for row in puzzle.strip().split()], np.uint8)
        return board

それでは、次のパズルを解いてみましょう。

In [13]:
board = load_puzzle('data/numberlink01.txt')
print(board)

[[1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 5 0]
 [0 6 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 4 3 0 0 0]
 [0 0 0 0 0 0 6 0 5 0]
 [0 8 0 3 0 0 0 0 0 0]
 [0 0 0 4 7 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 8 0]
 [0 7 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2]]


## 解き方

ナンバーリンクパズルを`add_circuit()`を用いて解く方法を説明します。このパズルでは、同じ数字のペアをつなぐルートを見つける必要がありますが、全てのマスを通る必要はありません。そこで、次のような方法を採用します。

1. **数字のペアごとに1つの回路を対応させる**  
   - すべての頂点（マス）を通る必要がないため、 **自己ループ** `(頂点番号, 頂点番号, 頂点変数)` を追加します。これにより、特定の頂点がルートから除外されることを許可できます。  
   - 数字のペアに対応する頂点番号を`n1`と`n2`とした場合、これらを必ず結ぶための辺`(n2, n1, True)`を追加します。

2. **各頂点は1つの回路にのみ属する制約を追加する**  
   - 各マスには複数の数字のペアが通る可能性があるため、それぞれのペアに対して異なる回路を割り当てます。  
   - しかし、1つのマスは1つの回路にしか属せないため、自己ループの変数が `True` の場合、そのマスはその数字の回路に含まれません。そのため、自己ループの変数のうち **1つだけを `False` にする制約** を追加します。これにより、あるマスが **1つの回路にのみ割り当てられ、それ以外の回路には含まれない** ことを保証します。

次の`create_nodes_edges(board)`は盤面 (board) を受け取り、盤面上の各マスに対応するインデックス情報 (nodes) と、それぞれのマスを隣接するマスと結ぶ辺 (edges) を作成します。次の二つデータを返します。

- `nodes`：各マスのインデックスを持つ2次元配列  
- `edges`：隣接するマス同士を結ぶ辺のリスト  

In [ ]:
def create_nodes_edges(board):
    h, w = board.shape
    nodes = np.arange(w * h).reshape((h, w))
    edges = []
    for r in range(h):
        for c in range(w):
            node = nodes[r, c]
            if c + 1 < w:
                right = nodes[r, c + 1]
                edges.extend([(node, right), (right, node)])
            if r + 1 < h:
                bottom = nodes[r + 1, c]
                edges.extend([(node, bottom), (bottom, node)])
    return nodes, edges

次の`solve_number_link(board)`関数でナンバーリンクパズルを解きます。

1. 各数字ごとに回路を作成
   - `number_starts`：各数字の開始地点（1つ目の位置）  
   - `layers_circuits`：各数字ごとの辺とブール変数の対応  
   - `layers_nodes`：各数字ごとの自己ループの変数  

2. 各マスに対する制約
   - 1つのマスは1つの回路にしか属せないようにする制約を`model.add_exactly_one()`で追加します。

3. 解を求め、結果を表示
   - ヘルプ関数`get_circuit()` を用いて各回路の経路を取得し、結果のグリッドを表示します。

In [ ]:
def solve_number_link(board):
    h, w = board.shape
    model = cp_model.CpModel()
    nodes, edges = create_nodes_edges(board)
    
    number_starts = {}
    layers_circuits = {}
    layers_nodes = {}
    
    for n in range(1, board.max() + 1):
        r, c = np.where(board == n)
        loc1 = r[0], c[0]
        loc2 = r[1], c[1]
        node1 = nodes[loc1]
        node2 = nodes[loc2]
    
        circuit_variables = [(s, t, model.new_bool_var(f"{n}_{s}_{t}")) for s, t in edges]
        node_variables = [(s, s, model.new_bool_var(f"{n}_{s}_{s}")) for s in nodes.ravel()]
    
        model.add_circuit(circuit_variables + node_variables + [(node2, node1, True)])
        number_starts[n] = node1
    
        layers_circuits[n] = circuit_variables
        layers_nodes[n] = node_variables
    
    for node in nodes.ravel():
        variables = [value[node][2] for value in layers_nodes.values()]
        model.add_exactly_one([~v for v in variables])
    
    solver = cp_model.CpSolver()
    solver.solve(model)

    result = np.zeros(w * h, dtype=np.uint8)
    for key, circuits in layers_circuits.items():
        solution = {(s, t):solver.value(v) for s, t, v in circuits}
        nodes = get_circuit(solution, start=number_starts[key])
        result[nodes] = key
    print(result.reshape(h, w))

In [19]:
%time solve_number_link(board)

[[1 6 6 6 6 6 6 6 6 6]
 [1 6 8 8 8 8 8 8 5 6]
 [1 6 8 1 3 3 3 8 5 6]
 [1 8 8 1 3 4 3 8 5 6]
 [1 8 1 1 3 4 6 8 5 6]
 [1 8 1 3 3 4 6 8 8 6]
 [1 1 1 4 7 4 6 6 8 6]
 [4 4 4 4 7 4 2 6 8 6]
 [4 7 7 7 7 4 2 6 6 6]
 [4 4 4 4 4 4 2 2 2 2]]
CPU times: total: 4.16 s
Wall time: 1.03 s
